In [1]:
#Imports correspondientes.
import urllib.request
import json
import difflib
from bottle import route, run, template

In [2]:
#Parametros de configuracion

#AÑADIR AQUÍ PARA SU FUNCIONAMIENTO
app_id=""
app_key=""


In [3]:

#FUNCIONALIDADES DEL SERVICIO REST

#Dado un plato, devuelve su receta.
def get_recipe(plate):
    plate=parse_blank_spaces(plate)
    contents = urllib.request.urlopen("https://api.edamam.com/search?app_id="+ app_id + "&app_key="+app_key+"&q=+"+plate).read()
    
    #contents es de tipo bytes. Es necesario pasarlo a json.
    contents = parse_bytes_to_JSON(contents)
    
    return get_best_hit(plate,contents)["recipe"]["ingredientLines"] #Devuelve una lista con los ingredientes sin procesar


def get_best_hit(plate,contentJSON):
    #Para cada posible match, buscamos el que tiene mayor similaridad.
    #La similaridad se busca mediante el encaje de cadenas. BUSCAR MEJORA.
    
    max=0    
    for x in contentJSON["hits"] :
        similarity=get_label_similarity(plate,x["recipe"]["label"])
        if(similarity>max):
            max=similarity
            best_hit=x
            
    return best_hit



In [4]:
#Métodos auxiliares

'''
    Dada una entrada (supuestamente el nombre de un plato), parsea su contenido sustituyendo los espacios
    en blanco por "%20".
    
    Esto se utiliza cuando el plato está compuesto por múltiples palabras (por ejemplo, tortilla de patata).
    En el caso de que el nombre del plato esté compuesto por una sola palabra (por ejemplo, lentejas), este método
    no hará nada.
    
    Para realizar una petición get, es necesario convertir:
        tortilla de patatas ==>  tortilla%20de%20patatas
        
    el resultado es un string parseado tal y como se indica arriba.
    
'''
def parse_blank_spaces(input):
    #Si no tenemos una sola palabra, parsea. En caso contrario, no hace nada.
    return input.replace(" ", "%20") 

'''
    Dado un objeto bytes, obtiene el json asociado.
    
    Se utiliza para formatear el resultado de las llamadas a APIs que devuelven un json.
    
    input: objeto tipo bytes a formatear.
    
    el resultado es un objeto json (diccionario)
    
'''    
def parse_bytes_to_JSON(input):
    decoded = input.decode('utf8') #Decodificamos usando utf-8. El resultado es un string con forma de json.
    return json.loads(decoded);  #Creamos el json a partir del string    


'''
    Obtiene la similaridad entre dos cadenas.
    
    Se usa para comparar las etiquetas de la receta que se solicita y las que encuentra el sistema.
    
    first: primera etiqueta a comparar.
    second: etiqueta que se comparara a first.

    el resultado es un porcentaje.

'''
def get_label_similarity(first, second):
    #Comparará las dos entradas en minuscula.
    seq = difflib.SequenceMatcher(None,first.lower(),second.lower())
    return seq.ratio()*100
    

In [ ]:
#ESQUELETO DEL SERVICIO REST

@route('/recipe/<plate>')
def index(plate):
    #return get_recipe(plate)
    return str(get_recipe(plate))

    
run(host='localhost', port=8080)

Bottle v0.12.16 server starting up (using WSGIRefServer())...
Listening on http://localhost:8080/
Hit Ctrl-C to quit.

127.0.0.1 - - [10/Mar/2019 15:46:14] "GET /recipe/tortilla%20de%20patatas HTTP/1.1" 200 294


In [ ]:
print(get_label_similarity("MUNDO","Mundo"))